In [ ]:
import numpy as np, matplotlib.pyplot as plt
from scipy.integrate import odeint
from matplotlib import Animation

plt.style.use(['science','notebook','grid', 'dark_background'])

For a collection of masses: $$\frac{d^2 \vec r_i}{dt^2}=\sum_{j \neq i}\frac{Gm_j}{r^3_{ij}}\vec r_{ij}$$
where
- $\vec r_i$ is the position of the ith particle and $m_j$ is the mass of the jth particle
- $\vec r_{ij}=\vec r_j-\vec r_i$

Converting to dimensionless quantities for python: $$\frac{d^2 \vec r'_i}{dt'^2}=\sum_{j \neq i}\frac{Gm'_j}{r'^3_{ij}}\vec r'_{ij}$$
where
- $\vec r'_i=\vec r_i/L$ and $L$ is some charachteristic length
- $m'_j=m_j/M$ and $M$ is some charachteristic mass
- $t'=t\sqrt{GM/L^3}$

# A Simple Start: 2-Body Problem

Set
- $M \equiv M_E=5.97e+24$ Kilograms
- $L \equiv 1\,AU=1.5e+11$ Meters

In [ ]:
M_E = 5.97e+24

m1 = M_E
m2 = 333000 * M_E

x1_0, y1_0, x2_0, y2_0 = 1, 0, 0, 0
vx1_0, vy1_0, vx2_0, vy2_0 = 0, np.sqrt(m2), 0, 0